In [ ]:
# 0. LIBRERÍAS

import pandas as pd
import numpy as np

In [ ]:
# 1. CARGA DEL CSV 

BegInv = r'd:\Usuario\Escritorio\HENRY 2025\PF\dataset desde kaggle\archive\BegInvFINAL12312016.csv'   #BegInv es Beginer Inventory, es el inventario inicial
df_beg = pd.read_csv(BegInv) 

print(f"✅ Archivo '{BegInv}' cargado correctamente.")
print(f"Dimensiones iniciales: {df_beg.shape}")

✅ Archivo 'd:\Usuario\Escritorio\HENRY 2025\PF\dataset desde kaggle\archive\BegInvFINAL12312016.csv' cargado correctamente.
Dimensiones iniciales: (206529, 9)


In [5]:
# 2. ANÁLISIS DE ESTRUCTURA Y NULOS 

print("\n Info del DataFrame")
df_beg.info()

print("\n Conteo de Nulos por Columna")
null_counts = df_beg.isnull().sum()
print(null_counts[null_counts > 0]) # Solo mostramos las que tienen problemas


 Info del DataFrame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206529 entries, 0 to 206528
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InventoryId  206529 non-null  object 
 1   Store        206529 non-null  int64  
 2   City         206529 non-null  object 
 3   Brand        206529 non-null  int64  
 4   Description  206529 non-null  object 
 5   Size         206529 non-null  object 
 6   onHand       206529 non-null  int64  
 7   Price        206529 non-null  float64
 8   startDate    206529 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 14.2+ MB

 Conteo de Nulos por Columna
Series([], dtype: int64)


In [6]:
# 3. CONVERSIÓN DE TIPOS 
# 'startDate' está como object (texto), lo pasamos a datetime
if df_beg['startDate'].dtype == 'object':
    df_beg['startDate'] = pd.to_datetime(df_beg['startDate'])
    print("\n✅ Columna 'startDate' convertida a datetime.")




✅ Columna 'startDate' convertida a datetime.


In [7]:
# 4. TRATAMIENTO DE NULOS (preparado para Automatización: el archivo actual dio cero nulos, pero en el futuro podría llegar un archivos con nulos)
# Caso común en este dataset: La columna 'City' suele tener nulos, pero 'Store' no.
# ESTRATEGIA: Imputación basada en el Store.
# Si la Tienda 1 es "Hardersfield", rellenamos los nulos de la Tienda 1 con ese valor.

if 'City' in df_beg.columns and df_beg['City'].isnull().sum() > 0:  #solo entra a este bloque si encuentras nulos en columna 'City'(y que exista una columna 'City').
    nulos_antes = df_beg['City'].isnull().sum()                     #guardamos los errores en esta variable
    
    # Rellenamos City usando el valor más frecuente (moda) de cada Store
    df_beg['City'] = df_beg.groupby('Store')['City'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "Unknown"))
    
    nulos_despues = df_beg['City'].isnull().sum()
    print(f"\n✅ Imputación de 'City' realizada: Se recuperaron {nulos_antes - nulos_despues} valores usando el mapeo de Tiendas.")

# Si quedara algún nulo (ej: una tienda nueva sin ciudad registrada), rellenamos con 'Unknown'
df_beg.fillna({'City': 'Unknown', 'Description': 'Sin Descripción'}, inplace=True)

In [8]:
# 5. EXPORT DEL ARCHIVO LIMPIO 
output_name = 'BegInvFINAL12312016_clean.csv'           # Usamos el sufijo "_clean" para diferenciarlo del original crudo.
df_beg.to_csv(output_name, index=False)         #Guardamos los datos en el disco duro, index=False: Instrucción vital para que no guarde la numeración de filas (0, 1, 2...)        
print(f"\n💾 Archivo exportado exitosamente: {output_name}")            #Confirmación visual para el usuario


💾 Archivo exportado exitosamente: BegInvFINAL12312016_clean.csv
